# Initial Setup

In [1]:
import os
import glob
import gradio as gr
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# API creds
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "type-your-api-key-here")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "type-your-api-key-here")
OLLAMA_API_KEY = "ollama"

GOOGLE_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
OLLAMA_BASE_URL = "http://localhost:11434/v1"

client_openai = OpenAI()
client_google = OpenAI(api_key=GOOGLE_API_KEY, base_url=GOOGLE_BASE_URL)
client_ollama = OpenAI(api_key=OLLAMA_API_KEY, base_url=OLLAMA_BASE_URL)

# Models
gpt = "gpt-4o-mini"
gemini = "models/gemini-2.0-flash"
llama = "llama3.2:latest"

# Build Context from Knowledge Base (KB)

In [3]:
# Paths
ABSOLUTE_PATH = os.path.abspath(os.getcwd())
KB_PATH = os.path.join(ABSOLUTE_PATH, "knowledge-base-nexatech")

def set_path(dir_name: str, kb_path: str=KB_PATH) -> str:
    return os.path.join(KB_PATH, dir_name)

COMPANY_DATA_PATH = set_path("company")
PRODUCT_DATA_PATH = set_path("products")

In [4]:
def get_file_name(file_path: str) -> str:
    """Get file name without extension format"""
    return Path(os.path.basename(file_path)).stem

def get_file_content(file_path: str) -> str:
    """Get file content"""
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()
    return content

context = {}

# Get company data
company_info = glob.glob(COMPANY_DATA_PATH + "/*" + ".md") # get all .md files
for info in company_info:
    title = get_file_name(info)
    context[title] = get_file_content(info)

# Get product data
products = glob.glob(PRODUCT_DATA_PATH + "/*" + ".md")
for product in products:
    prod_name = get_file_name(product)
    context[prod_name] = get_file_content(product)

print(context.keys())

dict_keys(['about', 'careers', 'overview', 'CustomerVault CRM', 'DataMind Analytics', 'ProjectFlow Pro', 'TeamSync Hub'])


# Simple RAG (Retrieval-Augmented Generation)

In [5]:
def get_relevant_context(message: str) -> list:
    "Get relevant context based on input message"
    relevant_context = []
    for context_title, context_details in context.items():
        if context_title in message or context_title.lower() in message:
            relevant_context.append(context_details)
    return relevant_context

# Testing
test = get_relevant_context("Tell me about spencer and datamind analytics")
len(test)

2

In [8]:
def add_context(message: str) -> str:
    """Add relevant context to input messages"""
    relevant_context = get_relevant_context(message)
    if not relevant_context:
        return message
    
    message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
    message += "\n\n".join(relevant_context)

    return message

# Testing
test = add_context("Tell me about datamind analytics")
print(test[:150])

Tell me about datamind analytics

The following additional context might be relevant in answering this question:

Company Profile
NexaTech Solutions i


# Prompting

In [9]:
system_prompt = "You are a specialist in providing precise and reliable information about NexaTech Solutions \
— a company offering cloud-based enterprise solutions and AI-driven business tools. \
Keep your answers concise and factual. If you lack the necessary context to answer, state that clearly. \
Do not fabricate any information."

def chat(message, history, client: OpenAI=client_ollama, model: str=llama):
    """Dedicated Gradio function for chatbot"""
    messages = [{"role": "system", "content": system_prompt}] + history
    
    # Add context using simple RAG
    message = add_context(message)
    messages.append({"role": "user", "content": message})

    # Generate response
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.8,
        stream=True
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

# UI

In [10]:
view = gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7905
* To create a public link, set `share=True` in `launch()`.
